## Logistic Regression on Copy Number Variation and RNAseq

In [1]:
import pandas as pd

In [83]:
#Import Data
y = pd.read_table('/Users/a.su/Documents/Cancer_RNAseq_CNV/CancerTypes_y.txt', sep = '\t', header = None)
x_cnv = pd.read_table('/Users/a.su/Documents/Cancer_RNAseq_CNV/CNV_processed.txt', sep = '\t', header = 0)
x_rna = pd.read_table('/Users/a.su/Documents/Cancer_RNAseq_CNV/RNAseq_processed.txt', sep = '\t', header = 0)

In [84]:
# Remove GeneID Column
x_cnv = x_cnv.drop('GeneID', axis = 1)
x_rna = x_rna.drop('GeneID', axis = 1)
# Transpose
x_cnv = x_cnv.transpose()
x_rna = x_rna.transpose()
print('x_cnv shape is:', x_cnv.shape)
print('x_rna shape is:', x_rna.shape)
print('y shape is:', y.shape)

x_cnv shape is: (668, 26094)
x_rna shape is: (668, 26094)
y shape is: (668, 1)


#### DropNa from CNV data

In [41]:
#Merge CNV x and y data
cnv = x_cnv.reset_index(drop = True)
cnv['y'] = y[0]

#Drop NaN
cnv_clean = cnv.dropna('index')
print(cnv_clean.shape)

#Unmerge CNV x and y data
y_cnv_clean = cnv_clean['y']
x_cnv_clean = cnv_clean.drop(['y'], axis = 1)

(542, 26095)


#### Impute CNV data

In [37]:
import numpy as np
from sklearn.impute import SimpleImputer
impute_median = SimpleImputer(strategy = 'median')

#Impute 
y_cnv_imputed = cnv['y']
x_cnv_imputed = cnv.drop(['y'], axis = 1)
x_cnv_imputed = impute_median.fit_transform(x_cnv_imputed)

#Check for NaN values
np.isnan(x_cnv_imputed).all()

False

In [54]:
x_rna[798].sum()

0.0

In [67]:
x_rna.std().value_counts()[0]

111

In [68]:
x_rna.sum().value_counts()[0]

111

#### Scale RNASeq and CNV data

In [71]:
def zscore(x):
    if x.std()==0:
        return x
    else:
        return (x-x.mean())/ x.std()
    
x_cnv_imputed = pd.DataFrame(x_cnv_imputed)

In [85]:
x_rna_processed = x_rna.transform(zscore)
y_rna_processed = y
x_cnv_processed = x_cnv_imputed.transform(zscore)
y_cnv_processed = y

In [73]:
#Visualise
x_rna_processed[np.random.choice(x_rna_processed.columns.values, size = 10, replace = False)].describe()

,25031,24711,20339,2042,25556,10202,23390,22091,23313,6263
count,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,668.0,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02
mean,-3.015552e-15,-9.588139e-16,-2.514206e-16,4.250592e-16,0.0,-4.748364e-16,-6.024788e-17,1.035433e-15,-4.605223e-16,-1.921284e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.799751e+00,-6.415402e-01,-6.989218e-01,-2.252745e-01,0.0,-5.323731e-01,-4.673270e-01,-1.950457e-01,-2.055896e+00,-1.227749e+00
25%,-7.110510e-01,-4.267988e-01,-5.521124e-01,-2.252745e-01,0.0,-5.323731e-01,-4.153762e-01,-1.950457e-01,-6.453704e-01,-6.919606e-01
50%,-1.868432e-01,-2.677942e-01,-3.779924e-01,-2.046518e-01,0.0,-4.784667e-01,-2.826844e-01,-1.950457e-01,-2.206979e-01,-2.232506e-01
75%,4.840146e-01,7.589458e-02,9.986880e-02,-1.187687e-01,0.0,1.203361e-01,-3.156066e-02,-1.950457e-01,4.731923e-01,3.362896e-01
max,6.017927e+00,1.452851e+01,8.569220e+00,1.710277e+01,0.0,8.202107e+00,1.175696e+01,1.701178e+01,1.112505e+01,7.748902e+00


In [74]:
x_cnv_processed[np.random.choice(x_cnv_processed.columns.values, size = 10, replace = False)].describe()

,78,19284,24391,16634,6411,17000,17307,11473,14274,10195
count,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02
mean,-2.887743e-16,1.966989e-16,9.972063e-19,-3.722904e-17,1.018812e-16,4.088546e-17,1.519078e-16,-3.732876e-16,5.750556e-17,-6.574914e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.928529e+00,-3.034655e+00,-3.331978e+00,-2.275024e+00,-3.236881e+00,-2.997208e+00,-2.441742e+00,-2.303250e+00,-4.525349e+00,-3.078891e+00
25%,-6.608156e-01,-6.275523e-01,-6.244311e-01,-6.795403e-01,-7.322270e-01,-6.123954e-01,-6.320894e-01,-6.166847e-01,-5.574981e-01,-6.375975e-01
50%,9.381721e-02,-4.613094e-02,1.950555e-02,5.168912e-02,5.003070e-02,-6.573807e-02,-1.258360e-01,4.618720e-02,-6.976906e-02,-1.157961e-01
75%,5.894547e-01,6.554109e-01,5.478158e-01,4.705373e-01,6.332291e-01,4.440966e-01,5.491144e-01,4.055298e-01,6.227346e-01,6.345004e-01
max,4.417968e+00,3.233232e+00,5.417243e+00,4.524598e+00,3.660945e+00,4.379963e+00,3.542402e+00,7.628642e+00,3.365628e+00,4.144223e+00


In [75]:
#Check for NaN values
print(x_rna_processed.isnull().any().any())
print(x_cnv_processed.isnull().any().any())

False
False


### Logistic Regression on CNV and RNAseq Individually (Unprocessed Data)

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [105]:
#Split data into training and test sets
x_cnv_train, x_cnv_test, y_cnv_train, y_cnv_test = train_test_split(x_cnv_clean, y_cnv_clean.values.flatten(), test_size = 0.25, random_state = 0)
x_rna_train, x_rna_test, y_rna_train, y_rna_test = train_test_split(x_rna, y.values.flatten(), test_size = 0.25, random_state = 0)

CNV_regression = LogisticRegression(penalty = 'l1', solver = 'liblinear')
RNA_regression = LogisticRegression(penalty = 'l1', solver = 'liblinear')

#Fit train set
CNV_regression.fit(x_cnv_train, y_cnv_train)
RNA_regression.fit(x_rna_train, y_rna_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [107]:
print('CNV accuracy:', CNV_regression.score(x_cnv_test, y_cnv_test))
print('RNA accurary:', RNA_regression.score(x_rna_test, y_rna_test))

CNV accuracy: 0.7058823529411765
RNA accurary: 0.7784431137724551


### Logistic Regression on CNV and RNAseq Individually (Processed Data)
Processing include imputation and scaling

In [103]:
#Split data into training and test sets
x_cnv_processed_train, x_cnv_processed_test, y_cnv_processed_train, y_cnv_processed_test = train_test_split(
    x_cnv_processed, y_cnv_processed.values.flatten(), test_size = 0.25, random_state = 0)
x_rna_processed_train, x_rna_processed_test, y_rna_processed_train, y_rna_processed_test = train_test_split(
    x_rna_processed, y_rna_processed.values.flatten(), test_size = 0.25, random_state = 0)

CNV_processed_regression = LogisticRegression(penalty = 'l1', solver = 'liblinear')
RNA_processed_regression = LogisticRegression(penalty = 'l1', solver = 'liblinear')

#Fit train set
CNV_processed_regression.fit(x_cnv_processed_train, y_cnv_processed_train)
RNA_processed_regression.fit(x_rna_processed_train, y_rna_processed_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [104]:
print('CNV processed accuracy:', CNV_processed_regression.score(x_cnv_processed_test, y_cnv_processed_test))
print('RNA processed accurary:', RNA_processed_regression.score(x_rna_processed_test, y_rna_processed_test))

CNV processed accuracy: 0.7305389221556886
RNA processed accurary: 0.8323353293413174


### Logistic Regression on combined CNV and RNAseq data (processed)

In [95]:
x_combined = pd.concat([x_cnv_processed.reset_index(drop = True), x_rna_processed.reset_index(drop = True)] , axis = 1)
y_combined = y[0]
combined.shape

(668, 52189)

In [96]:
combined.head()

,0,1,2,3,4,5,6,7,8,9,...,26085,26086,26087,26088,26089,26090,26091,26092,26093,y
0,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,-0.523163,...,0.107225,0.946796,1.244745,-0.238482,0.735332,-0.360789,0.233145,-0.310503,-0.811940,0
1,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,-0.233295,...,-0.093206,0.323596,0.753448,1.486041,-0.715673,0.053541,-0.570047,0.106635,-0.864980,0
2,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,-0.668243,...,1.042499,0.820289,-0.676087,0.413422,-1.037367,-0.375897,0.183390,0.019250,-0.309738,0
3,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,-1.929221,...,1.440989,0.739403,-0.680702,0.789668,-0.079546,-0.074477,-0.363443,0.044287,-0.399333,0
4,1.069856,1.069856,1.069856,1.069856,1.069856,1.069856,1.069856,1.069856,1.069856,1.069856,...,-0.075358,-0.869696,-0.126601,-0.016079,-0.637967,-0.297226,-0.488207,1.142189,-0.694951,0


In [94]:
#Visualise
combined[np.random.choice(combined.columns.values, size = 5, replace = False)].describe()

,6099,6099,10497,10497,12996,12996,11405,11405,18427,18427
count,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02,6.680000e+02
mean,-6.355528e-16,1.645806e-16,-3.024859e-16,-1.225733e-18,3.556703e-17,-1.246508e-18,-2.504650e-16,2.002556e-15,1.329608e-18,-2.953974e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.566857e+00,-3.170550e-01,-3.165486e+00,-4.501642e-02,-5.241609e+00,-1.277855e+00,-3.690970e+00,-1.008753e+00,-2.202390e+00,-1.689011e+00
25%,-6.747563e-01,-2.839991e-01,-5.754377e-01,-4.501642e-02,-6.176092e-01,-6.447860e-01,-6.603163e-01,-4.686475e-01,-6.203517e-01,-7.496393e-01
50%,6.686310e-02,-2.382796e-01,-6.178360e-02,-4.501642e-02,5.319860e-02,-2.476919e-01,-7.631572e-02,-1.656951e-01,-8.095864e-02,-1.708477e-01
75%,6.105370e-01,-1.291747e-01,6.873569e-01,-4.501642e-02,5.747659e-01,3.259826e-01,5.966636e-01,2.199472e-01,5.451191e-01,5.218052e-01
max,4.108496e+00,1.625288e+01,3.986695e+00,2.576135e+01,3.342930e+00,8.256115e+00,4.310587e+00,1.723828e+01,4.068646e+00,6.306747e+00


In [111]:
#Split data into training and test sets
x_combined_train, x_combined_test, y_combined_train, y_combined_test = train_test_split(
    x_combined, y_combined.values.flatten(), test_size = 0.25, random_state = 0)

Combined_regression = LogisticRegression(penalty = 'l1',solver = 'liblinear')
Combined_regression.fit(x_combined_train, y_combined_train)
Combined_regression.score(x_combined_test, y_combined_test)

0.844311377245509

#### Grid Search

In [113]:
from sklearn.model_selection import GridSearchCV

In [114]:
parameters = {'C': [0.001, 0.01, 1, 10, 100, 1000]}
clf = GridSearchCV( estimator = Combined_regression, param_grid = parameters)

In [115]:
clf.fit(x_combined_train, y_combined_train)

/Users/a.su/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [116]:
clf.best_score_

0.810379241516966

In [117]:
clf.score(x_combined_test, y_combined_test)

0.8502994011976048

In [118]:
clf.best_params_

{'C': 1}

#### Elastic Net

In [119]:
from sklearn.linear_model import SGDClassifier

In [137]:
SGD = SGDClassifier(alpha = 0.1, epsilon = 0.01, loss = 'log', max_iter = 100, penalty = 'elasticnet')

In [138]:
SGD.fit(x_combined_train, y_combined_train)

SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=True, epsilon=0.01, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=100,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.25, verbose=0, warm_start=False)

In [139]:
SGD.score(x_combined_test, y_combined_test)

0.7664670658682635

In [143]:
SGD_parameters = {'alpha': [0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'penalty':['elasticnet'], 
                  'epsilon': [0.001, 0.01, 0.1, 1, 10]}

In [144]:
SGD_search = GridSearchCV(estimator = SGD, param_grid= SGD_parameters)

In [145]:
SGD_search.fit(x_combined_train, y_combined_train)

/Users/a.su/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/a.su/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'penalty': ['elasticnet'], 'epsilon': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [146]:
SGD_search.score(x_combined_test, y_combined_test)

0.7941176470588235

In [147]:
SGD_search.best_params_

{'alpha': 0.1, 'epsilon': 0.01, 'penalty': 'elasticnet'}